# Задание 3
## Статистика бросков игроков НБА. 
Игрок попадает с вероятностью р - исход броска имеет распределение Бернулли. 
Распределение числа попаданий в последовательности из n независимых бросков - биномиальное.

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
import scipy.stats as st
from statsmodels.stats.proportion import binom_test
import seaborn as sns
from statsmodels.stats.proportion import proportion_confint

In [70]:
data = pd.read_csv("nba.csv", sep = ',')
atm = [] 
score = []
atm.append(data.iloc[:,3])
atm.append(data.iloc[:,4])
atm = pd.concat(atm, axis=1)
score.append(data.iloc[:,5])
score.append(data.iloc[:,6])
score = pd.concat(score, axis=1)
atm = np.array(atm)
score = np.array(score)

Так как нет четкой связи между бросками одного игрока в домашних и гостевых играх, то считаем выборки независимыми.
Для каждого игрока используем Z-критерий для разности двух долей, независимые выборки. 
H_0 : данный игрок забивает одинаково хорошо дома и в гостях.
H_1 : игрок попадает в кольцо лучше дома.

In [71]:
from statsmodels.stats.proportion import proportions_ztest
pvals  = []
stats = []
for i in range (936): 
    a = score[i,:]
    b = atm[i,:]
    stat, pval = proportions_ztest(a, b, alternative = 'smaller' )
    stats.append(stat)
    pvals.append(pval)  

In [72]:
pvals = np.array(pvals)

Теперь сделаем множественную проверку гипотез методами Холма и Шидака-Холма. Заметим, что статистики для каждого игрока независимы.

In [73]:
from statsmodels.stats.multitest import multipletests
rejects = (multipletests(pvals, alpha = 0.05, method = 'holm')[0])
print ('Ho: true, multiple_test = Holm; Ho not rejected:', 936 - rejects[:936].sum())
rejects = (multipletests(pvals, alpha = 0.05, method = 'holm-sidak')[0])
print ('Ho: true, multiple_test = Holm-Sidak; Ho not rejected:', 936 - rejects[:936].sum())

Ho: true, multiple_test = Holm; Ho not rejected: 936
Ho: true, multiple_test = Holm-Sidak; Ho not rejected: 936


Таким образом, данные не противоречат нулевой гипотезе - "нет разницы в успехе бросков у игроков в домашних и гостевых игра".